# Lab 3 - Memory Process Analysis Deeper Dive
## Justin Vermillion
### 3 November 2025

In [1]:
dumpPath = "iveep/CTICOE-iveep_374_2025-03-17T17-30Z.mem"

In [2]:
# load my VirusTotal API key so I can check file hashes later
from dotenv import load_dotenv
import os

load_dotenv()

vtApiKey = os.getenv("VT_API_KEY")

First take a hash of the dump file. Normally would compare this with the hash from the person who gathered the dump to ensure no tampering, but one was not provided with this file.

In [3]:
! md5sum {dumpPath}

59526a220e839ac8139fec04539baeac  iveep/CTICOE-iveep_374_2025-03-17T17-30Z.mem


First step is to see which processes were running, using the pslist plugin in volatility3.

In [4]:
! vol -f {dumpPath} windows.pslist

Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

4	0	System	0x840e90261040	213	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	Disabled
108	4	Registry	0x840e9037c040	4	-	N/A	False	2025-03-12 16:39:25.000000 UTC	N/A	Disabled
536	4	smss.exe	0x840e910b4040	2	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	Disabled
804	796	csrss.exe	0x840e921650c0	12	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
880	796	wininit.exe	0x840e9228c080	1	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
888	872	csrss.exe	0x840e92291140	10	-	1	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
980	872	winlogon.exe	0x840e92305080	3	-	1	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
1000	880	services.exe	0x840e9230b100	7	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
548	880	lsass.exe	0x840e92349080	10	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled

A couple of items that stand out to me right away are:
- PSEXESVC.exe: the remote execution service for PsExec
- excel-update.exe: this is a fishy filename that sounds like a fake updater

Next I will check the relationships between processes with pstree.

In [5]:
! vol -f {dumpPath} windows.pstree | cut -d$'\t' -f1,2,3

Volatility 3 Framework 2.26.2ng finished                        

PID	PPID	ImageFileName

4	0	System
* 536	4	smss.exe
* 1924	4	MemCompression
* 108	4	Registry
804	796	csrss.exe
880	796	wininit.exe
* 1000	880	services.exe
** 11776	1000	svchost.exe
** 3604	1000	svchost.exe
** 4636	1000	svchost.exe
** 11296	1000	PSEXESVC.exe
*** 4672	11296	excel-update.e
**** 13752	4672	cmd.exe
***** 12392	13752	conhost.exe
***** 13156	13752	powershell.exe
** 1580	1000	svchost.exe
*** 15784	1580	taskhostw.exe
*** 6380	1580	taskhostw.exe
*** 4428	1580	taskhostw.exe
*** 10476	1580	taskhostw.exe
** 2092	1000	svchost.exe
** 1076	1000	svchost.exe
** 3636	1000	svchost.exe
** 7220	1000	svchost.exe
** 4156	1000	svchost.exe
** 3648	1000	CSFalconServic
*** 6624	3648	CSFalconContai
*** 324	3648	CSFalconContai
*** 1192	3648	CSFalconContai
*** 6604	3648	CSFalconContai
*** 5980	3648	CSFalconContai
** 1604	1000	svchost.exe
** 8772	1000	svchost.exe
** 1612	1000	svchost.exe
** 604	1000	svchost.exe
** 1124	1000	svchost.exe

We can see that excel-update.exe was started by PSEXESVC.exe, which indicates it was executed remotely. It then spawned a cmd prompt which was used to spawn a powershell session. Seems suspicious to me.

- 11296	1000	PSEXESVC.exe
    - 4672	11296	excel-update.exe
        - 13752	4672	cmd.exe
            - 12392	13752	conhost.exe
            - 13156	13752	powershell.exe

It also looks like there was a spoolsv.exe spawned by a powershell.exe, which is pretty suspicious since I would expect the print spooler would've been spawned by services.exe (and it does appear to be, earlier in the tree).

- 9940	2804	userinit.exe
    - 9852	9940	explorer.exe
        - 15172	9852	powershell.exe
            - 13176	15172	conhost.exe
            - 14536	15172	spoolsv.exe

Next I will use the dlllist plugin to examine excel-update.exe and the fake spoolsv.exe

In [6]:
! vol -f {dumpPath} windows.dlllist --pid=4672

Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
PID	Process	Base	Size	Name	Path	LoadTime	File output

4672	excel-update.e	0x400000	0x16000	excel-update.exe	C:\Windows\excel-update.exe	2025-03-17 17:01:18.000000 UTC	Disabled
4672	excel-update.e	0x7ffb50170000	0x1f8000	ntdll.dll	C:\Windows\SYSTEM32\ntdll.dll	2025-03-17 17:01:18.000000 UTC	Disabled
4672	excel-update.e	0x7ffb4edc0000	0x59000	wow64.dll	C:\Windows\System32\wow64.dll	2025-03-17 17:01:18.000000 UTC	Disabled
4672	excel-update.e	0x7ffb4ed10000	0x83000	wow64win.dll	C:\Windows\System32\wow64win.dll	2025-03-17 17:01:18.000000 UTC	Disabled
4672	excel-update.e	0x77260000	0xa000	wow64cpu.dll	C:\Windows\System32\wow64cpu.dll	2025-03-17 17:01:18.000000 UTC	Disabled
4672	excel-update.e	0x640000	0x16000	umppc19205.dll	C:\Windows\System32\umppc19205.dll	2025-03-17 17:01:19.000000 UTC	Disabled
4672	excel-update.e	0x400000	0x16000	excel-update.exe	C:\Windows\excel-update.exe	1678-08-08 10:10:11.

For excel-update.exe, there is a lot going on here that is suspicious.

1. The file is located in C:\Windows. A legitimate Office updater would not be stored here.
2. Timestamps have been tampered showing impossible years (1600s-1700s).
3. It has loaded quite a few networking DLLs, including WININET.dll, WINHTTP.dll, DNSAPI.dll, etc.

In [7]:
! vol -f {dumpPath} windows.dlllist --pid=14536

Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
PID	Process	Base	Size	Name	Path	LoadTime	File output

14536	spoolsv.exe	0x7ff7eb690000	0xd2000	spoolsv.exe	C:\Windows\System32\spoolsv.exe	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb50170000	0x1f8000	ntdll.dll	C:\Windows\SYSTEM32\ntdll.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4fac0000	0xbd000	KERNEL32.DLL	C:\Windows\System32\KERNEL32.DLL	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4dd20000	0x2f6000	KERNELBASE.dll	C:\Windows\System32\KERNELBASE.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4f180000	0x19f000	USER32.dll	C:\Windows\System32\USER32.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4d990000	0x22000	win32u.dll	C:\Windows\System32\win32u.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4e570000	0x2b000	GDI32.dll	C:\Windows\System32\GDI32.dll	2025-03-17 17:12:

This spoolsv.exe has some suspicious features as well.

1. Loads wininet.dll, used for network communication
2. Loads mscoree.dll, which is the .NET runtime--spoolsv is a native library and does not need this
3. Loads amsi.dll, the anti-malware scanning interface. I reckon this is being patched to bypass scanning detection.

Next I am going to dump the process executables so I can:

- Hash and check them against VT
- Upload to Any.Run for sandbox analysis

In [8]:
! mkdir dump/excel-update
! vol -f {dumpPath} -o dump/excel-update dumpfiles --pid=4672

mkdir: dump/excel-update: File exists
Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
Cache	FileObject	FileName	Result

DataSectionObject	0x840e96fa8770	apphelp.dll	file.0x840e96fa8770.0x840e9d6613d0.DataSectionObject.apphelp.dll-2.dat
ImageSectionObject	0x840e96fa8770	apphelp.dll	file.0x840e96fa8770.0x840e9757adc0.ImageSectionObject.apphelp.dll-2.img
DataSectionObject	0x840e920f5a30	locale.nls	Error dumping file
DataSectionObject	0x840ea19192c0	excel-update.exe	file.0x840ea19192c0.0x840e9d664850.DataSectionObject.excel-update.exe-2.dat
ImageSectionObject	0x840ea19192c0	excel-update.exe	file.0x840ea19192c0.0x840e90b2fd70.ImageSectionObject.excel-update.exe-2.img
DataSectionObject	0x840e920dfca0	umppc19205.dll	file.0x840e920dfca0.0x840ea708d650.DataSectionObject.umppc19205.dll-2.dat
ImageSectionObject	0x840e920dfca0	umppc19205.dll	file.0x840e920dfca0.0x840e923efcf0.ImageSectionObject.umppc19205.dll-2.img
DataSectionObject	0x840e92c24820	CsX

In [9]:
%%bash
cp dump/excel-update/file.0x840ea19192c0.0x840e90b2fd70.ImageSectionObject.excel-update.exe.img exes/excel-update.exe
file exes/excel-update.exe
md5sum exes/excel-update.exe

exes/excel-update.exe: PE32 executable for MS Windows 4.00 (GUI), Intel i386, 4 sections
6a4fc8cf4e980a57dc411c4d5a816e3e  exes/excel-update.exe


In [10]:
! curl -s -X POST   https://www.virustotal.com/vtapi/v2/file/scan -F apikey={vtApiKey} -F file=@exes/excel-update.exe

{"md5":"6a4fc8cf4e980a57dc411c4d5a816e3e","permalink":"f-e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39-1762169997","resource":"e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39","response_code":1,"scan_id":"e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39-1762169997","sha1":"38fe0be3cec10e0470f8ad8a1c9bce73913d8d31","sha256":"e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39","verbose_msg":"Scan request successfully queued, come back later for the report"}


In [11]:
! curl -s -X POST   https://www.virustotal.com/vtapi/v2/file/report -F apikey={vtApiKey} -F resource="e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39"

{"md5":"6a4fc8cf4e980a57dc411c4d5a816e3e","permalink":"https://www.virustotal.com/gui/file/e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39/detection/f-e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39-1762169110","positives":55,"resource":"e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39","response_code":1,"scan_date":"2025-11-03 11:25:10","scan_id":"e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39-1762169110","scans":{"ALYac":{"detected":true,"result":"Trojan.CryptZ.Marte.1.Gen","update":"20251103","version":"2.0.0.10"},"APEX":{"detected":true,"result":"Malicious","update":"20251101","version":"6.710"},"AVG":{"detected":true,"result":"Win32:Meterpreter-C [Trj]","update":"20251103","version":"23.9.8494.0"},"Acronis":{"detected":false,"result":null,"update":"20240328","version":"1.2.0.121"},"AhnLab-V3":{"detected":true,"result":"Trojan/Win32.Shell.R1283","update":"20251103","version":"3.29.0.10596"},"Alibaba":{"detected":fa

So we can see here that this is flagged by VirusTotal as trojan.swrort/cryptz.

In [12]:
! mkdir dump/spoolsv
! vol -f {dumpPath} -o dump/spoolsv dumpfiles --pid=14536

mkdir: dump/spoolsv: File exists
Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
Cache	FileObject	FileName	Result

DataSectionObject	0x840e9c35e980	winnlsres.dll.mui	Error dumping file
DataSectionObject	0x840ea1848180	KernelBase.dll.mui	file.0x840ea1848180.0x840e92d8e090.DataSectionObject.KernelBase.dll.mui-2.dat
DataSectionObject	0x840e920f5a30	locale.nls	Error dumping file
DataSectionObject	0x840e96af3dc0	winnlsres.dll	Error dumping file
DataSectionObject	0x840e96af0ee0	winnlsres.dll.mui	Error dumping file
DataSectionObject	0x840e920dfca0	umppc19205.dll	file.0x840e920dfca0.0x840ea708d650.DataSectionObject.umppc19205.dll-2.dat
ImageSectionObject	0x840e920dfca0	umppc19205.dll	file.0x840e920dfca0.0x840e923efcf0.ImageSectionObject.umppc19205.dll-2.img
DataSectionObject	0x840e92c1ae50	CsXumd64_19205.dll	Error dumping file
ImageSectionObject	0x840e92c1ae50	CsXumd64_19205.dll	file.0x840e92c1ae50.0x840e923e6a20.ImageSectionObject.CsXumd64_19205.

In [13]:
%%bash
cp dump/spoolsv/file.0x840e9684fc80.0x840e96b08010.ImageSectionObject.spoolsv.exe.img exes/spoolsv.exe
file exes/spoolsv.exe
md5sum exes/spoolsv.exe

exes/spoolsv.exe: PE32+ executable for MS Windows 10.00 (GUI), x86-64, 7 sections
a227f459b3cc14bfbc40d05c580227fd  exes/spoolsv.exe


In [14]:
! curl -s -X POST   https://www.virustotal.com/vtapi/v2/file/scan -F apikey={vtApiKey} -F file=@exes/spoolsv.exe

{"md5":"a227f459b3cc14bfbc40d05c580227fd","permalink":"f-9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669-1762170012","resource":"9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669","response_code":1,"scan_id":"9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669-1762170012","sha1":"1e6ff6d5b740273c73ae46d7e30348f1c3ec5f6f","sha256":"9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669","verbose_msg":"Scan request successfully queued, come back later for the report"}


In [15]:
! curl -s -X POST   https://www.virustotal.com/vtapi/v2/file/report -F apikey={vtApiKey} -F resource="9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669"

{"md5":"a227f459b3cc14bfbc40d05c580227fd","permalink":"https://www.virustotal.com/gui/file/9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669/detection/f-9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669-1762169123","positives":0,"resource":"9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669","response_code":1,"scan_date":"2025-11-03 11:25:23","scan_id":"9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669-1762169123","scans":{"ALYac":{"detected":false,"result":null,"update":"20251103","version":"2.0.0.10"},"APEX":{"detected":false,"result":null,"update":"20251101","version":"6.710"},"AVG":{"detected":false,"result":null,"update":"20251103","version":"23.9.8494.0"},"Acronis":{"detected":false,"result":null,"update":"20240328","version":"1.2.0.121"},"AhnLab-V3":{"detected":false,"result":null,"update":"20251103","version":"3.29.0.10596"},"Alibaba":{"detected":false,"result":null,"update":"20190527","version":"0.3.0.5"},"Antiy-AVL":{

This one is coming back as not malicious, but I still find it suspicious that it was launched from powershell.

In [16]:
! vol -f {dumpPath} windows.malware.malfind --pid 4672 15172 13156 14536 11296

Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm

11296	PSEXESVC.exe	0x1570000	0x1570fff	VadS	PAGE_EXECUTE_READWRITE	1	1	Disabled	N/A	
8b ff 55 8b ec e9 db 58 84 73 cc cc cc cc cc cc ..U....X.s......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 ..U.............
00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00 ..........-.....	8b ff 55 8b ec e9 db 58 84 73 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00
11296	PSEXESVC.exe	0x1580000	0x1580fff	VadS	PAGE_EXECUTE_READWRITE	1	1	Disabled	N/A	
8b ff 55 8b ec e9 6b de 78 73 cc cc cc cc cc cc ..U...k.xs......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00

There are a lot of RWX VADs, one in particular for excel-update.exe contains the WinPE executable's MZ header which is not so common outside of malware.

In [17]:
! vol -f {dumpPath} windows.handles --pid 4672 14536 15172 13156

Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
PID	Process	Offset	HandleValue	Type	GrantedAccess	Name

4672	excel-update.e	0x840e97fb28e0	0x4	Event	0x1f0003	-
4672	excel-update.e	0x840e97fb6f60	0x8	Event	0x1f0003	-
4672	excel-update.e	0x840e9818db50	0xc	WaitCompletionPacket	0x1	-
4672	excel-update.e	0x840e9f7876c0	0x10	IoCompletion	0x1f0003	-
4672	excel-update.e	0x840e96d9ed80	0x14	TpWorkerFactory	0xf00ff	-
4672	excel-update.e	0x840ea4ebbd90	0x18	IRTimer	0x100002	-
4672	excel-update.e	0x840e9818eed0	0x1c	WaitCompletionPacket	0x1	-
4672	excel-update.e	0x840ea4ebcc70	0x20	IRTimer	0x100002	-
4672	excel-update.e	0x840e9818f210	0x24	WaitCompletionPacket	0x1	-
4672	excel-update.e	0x840e9f78a1b0	0x28	EtwRegistration	0x804	-
4672	excel-update.e	0x840e9f78cad0	0x2c	EtwRegistration	0x804	-
4672	excel-update.e	0x840e9f798a70	0x30	EtwRegistration	0x804	-
4672	excel-update.e	0xb708012fede0	0x34	Directory	0x3	KnownDlls
4672	excel-update.e	0x840e97fb69e

One of the powershell instances has an open handle to a transcript file: PowerShell_transcript.IVEEP.Ld2S6HKt.20250317110630.txt

In [18]:
! vol -f {dumpPath} windows.privileges --pid 15172 13156 14536 4672

Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
PID	Process	Value	Privilege	Attributes	Description

4672	excel-update.e	2	SeCreateTokenPrivilege		Create a token object
4672	excel-update.e	3	SeAssignPrimaryTokenPrivilege	Present	Replace a process-level token
4672	excel-update.e	4	SeLockMemoryPrivilege	Present,Enabled,Default	Lock pages in memory
4672	excel-update.e	5	SeIncreaseQuotaPrivilege	Present	Increase quotas
4672	excel-update.e	6	SeMachineAccountPrivilege		Add workstations to the domain
4672	excel-update.e	7	SeTcbPrivilege	Present,Enabled,Default	Act as part of the operating system
4672	excel-update.e	8	SeSecurityPrivilege	Present	Manage auditing and security log
4672	excel-update.e	9	SeTakeOwnershipPrivilege	Present	Take ownership of files/objects
4672	excel-update.e	10	SeLoadDriverPrivilege	Present	Load and unload device drivers
4672	excel-update.e	11	SeSystemProfilePrivilege	Present,Enabled,Default	Profile system performance
4672	

All of the suspicious processes have very elevated privileges.

In [19]:
! vol -f {dumpPath} windows.netscan

Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
Offset	Proto	LocalAddr	LocalPort	ForeignAddr	ForeignPort	State	PID	Owner	Created

0x840e9028e470	TCPv4	0.0.0.0	49668	0.0.0.0	0	LISTENING	2904	spoolsv.exe	2025-03-12 16:40:03.000000 UTC
0x840e9028e890	TCPv4	0.0.0.0	445	0.0.0.0	0	LISTENING	4	System	2025-03-12 16:40:05.000000 UTC
0x840e9028e890	TCPv6	::	445	::	0	LISTENING	4	System	2025-03-12 16:40:05.000000 UTC
0x840e90e64010	TCPv4	10.20.20.23	50167	23.194.127.11	443	CLOSE_WAIT	4236	SearchApp.exe	2025-03-12 16:51:49.000000 UTC
0x840e90ed0500	UDPv4	10.20.20.23	138	*	0		4	System	2025-03-12 16:39:44.000000 UTC
0x840e90ed0820	UDPv4	10.20.20.23	137	*	0		4	System	2025-03-12 16:39:44.000000 UTC
0x840e911dd470	TCPv4	10.20.20.23	139	0.0.0.0	0	LISTENING	4	System	2025-03-12 16:39:44.000000 UTC
0x840e911dd730	TCPv4	0.0.0.0	135	0.0.0.0	0	LISTENING	1076	svchost.exe	2025-03-12 16:39:53.000000 UTC
0x840e911de650	TCPv4	0.0.0.0	49667	0.0.0.0	0	LISTENING	1580	svch

spoolsv.exe and excel-update.exe are both talking to the same IP. This definitely looks like C&C traffic.

| Offset          | Proto | Local IP     | Local Port | Remote IP   | Remote Port | State       | PID   | Process         | Timestamp                          |
|-----------------|-------|--------------|------------|-------------|-------------|-------------|-------|-----------------|-------------------------------------|
| 0x840e966cb010  | TCPv4 | 10.20.20.23  | 60023      | 35.86.58.69 | 80          | CLOSED      | 14536 | spoolsv.exe     | 2025-03-17 17:30:23.000000 UTC     |
| 0x840e97632010  | TCPv4 | 10.20.20.23  | 59990      | 35.86.58.69 | 80          | CLOSED      | 14536 | spoolsv.exe     | 2025-03-17 17:29:32.000000 UTC     |
| 0x840e982f34b0  | TCPv4 | 10.20.20.23  | 59231      | 35.86.58.69 | 4443        | ESTABLISHED | 4672  | excel-update.e  | 2025-03-17 17:01:19.000000 UTC     |
| 0x840e98f25b40  | TCPv4 | 10.20.20.23  | 60028      | 35.86.58.69 | 80          | CLOSED      | 14536 | spoolsv.exe     | 2025-03-17 17:30:35.000000 UTC     |
| 0x840e99d66520  | TCPv4 | 10.20.20.23  | 59329      | 35.86.58.69 | 4444        | CLOSED      | 14536 | spoolsv.exe     | 2025-03-17 17:12:10.000000 UTC     |
| 0x840e9b2dd010  | TCPv4 | 10.20.20.23  | 60030      | 35.86.58.69 | 80          | ESTABLISHED | 14536 | spoolsv.exe     | 2025-03-17 17:30:36.000000 UTC     |
| 0x840e9ce6c880  | TCPv4 | 10.20.20.23  | 60020      | 35.86.58.69 | 80          | CLOSED      | 14536 | spoolsv.exe     | 2025-03-17 17:30:21.000000 UTC     |
| 0x840e9dfba010  | TCPv4 | 10.20.20.23  | 60027      | 35.86.58.69 | 80          | CLOSED      | 14536 | spoolsv.exe     | 2025-03-17 17:30:32.000000 UTC     |
| 0x840e9ea7da20  | TCPv4 | 10.20.20.23  | 60026      | 35.86.58.69 | 80          | CLOSED      | 14536 | spoolsv.exe     | 2025-03-17 17:30:30.000000 UTC     |


Next I'm going to take a look at that powershell transcript.

In [20]:
! vol -f {dumpPath} -o dump/ windows.dumpfiles --virtaddr 0x840ea06132e0

Volatility 3 Framework 2.26.2
Progress:  100.00		PDB scanning finished                        
Cache	FileObject	FileName	Result

DataSectionObject	0x840ea06132e0	PowerShell_transcript.IVEEP.Ld2S6HKt.20250317110630.txt	file.0x840ea06132e0.0x840e9d64ced0.DataSectionObject.PowerShell_transcript.IVEEP.Ld2S6HKt.20250317110630.txt-2.dat
SharedCacheMap	0x840ea06132e0	PowerShell_transcript.IVEEP.Ld2S6HKt.20250317110630.txt	file.0x840ea06132e0.0x840e979eca90.SharedCacheMap.PowerShell_transcript.IVEEP.Ld2S6HKt.20250317110630.txt-2.vacb


In [21]:
! cat dump/file.0x840ea06132e0.0x840e9d64ced0.DataSectionObject.PowerShell_transcript.IVEEP.Ld2S6HKt.20250317110630.txt.dat

﻿**********************
Windows PowerShell transcript start
Start time: 20250317110650
Username: AzureAD\IanVeep
RunAs User: AzureAD\IanVeep
Configuration Name: 
Machine: IVEEP (Microsoft Windows NT 10.0.19045.0)
Host Application: C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe
Process ID: 15172
PSVersion: 5.1.19041.4474
PSEdition: Desktop
PSCompatibleVersions: 1.0, 2.0, 3.0, 4.0, 5.0, 5.1.19041.4474
BuildVersion: 10.0.19041.4474
CLRVersion: 4.0.30319.42000
WSManStackVersion: 3.0
PSRemotingProtocolVersion: 2.3
SerializationVersion: 1.1.0.1
**********************
**********************
Command start time: 20250317110755
**********************
PS C:\Windows\system32> New-ItemProperty -Path 'HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Run' -Value '"C:\Program Files\KeePass Password Safe 2\KeePass.exe"' -PropertyType 'String' -Name 'KeePass' -Force


KeePass      : "C:\Program Files\KeePass Password Safe 2\KeePass.exe"
PSPath       : Microsoft.PowerShell.Core\Registry::HKEY_L

The transcript shows a deliberate post-exploitation chain: the attacker creates Run-key persistence under HKLM to launch KeePass, then plants a second KeePass DLL and hides it, stuffs an alternate data stream named secret.exe into that DLL and reflectively loads it in memory, stages an extra payload to system32, and finally brings in KDU to abuse CVE-2015-2291 by loading the vulnerable NalDrv to map a “victim” signed driver (PROCEXP152.sys) and execute custom shellcode that loads an unsigned kernel driver (infinity_hook_pro_max.sys). That sequence gives kernel-level code execution, lets them tamper with tokens, drivers, and telemetry, and explains the crazy privileges we saw on those processes earlier.

### Potential IOCs

 Malicious Executables:
  - excel-update.exe (Meterpreter trojan - 55/72 AV detections)
    - MD5: 6a4fc8cf4e980a57dc411c4d5a816e3e
    - SHA1: 38fe0be3cec10e0470f8ad8a1c9bce73913d8d31
    - SHA256: e5f9d9a717cf801134934d67414f924164b9ae2de18ce370abe238d9b5edac39
      
  - spoolsv.exe (malicious - masquerading as Windows print spooler)
    - MD5: a227f459b3cc14bfbc40d05c580227fd
    - SHA1: 1e6ff6d5b740273c73ae46d7e30348f1c3ec5f6f
    - SHA256: 9d92880b0330090c803c9262eb6097b71081adee4cd6032ba9a54c00062fc669

  Malicious File Names

  Downloaded from C2:
  - KeePassLibC64.dll - Fake KeePass library
  - KeePassLibC64_1.dll - Fake KeePass library with ADS payload
  - APC.exe - Malicious payload
  - httpstage64.bin - HTTP stager
  - kdu.exe - Kernel Driver Utility (CVE-2015-2291 exploit)
  - infinity_hook_pro_max.sys - Malicious kernel driver
  - drv64.dll - Driver exploit database
  - NalDrv.sys - Vulnerable driver (CVE-2015-2291)
  - PROCEXP152.sys - Process Explorer driver (abused as victim)
  - PSEXESVC.exe - PsExec service

  IP Addresses

  - 35.86.58.69 - C2 server (port 8000, 80, 4443, 4444)
  - 10.20.20.23 - Compromised internal host

  File Paths

  - C:\Program Files\KeePass Password Safe 2\KeePassLibC64.dll
  - C:\Program Files\KeePass Password Safe 2\KeePassLibC64_1.dll
  - C:\Windows\system32\httpstage64.bin
  - C:\Users\IanVeep\Documents\PowerShell_transcript.IVEEP.Ld2S6HKt.20250317110630.txt
  - C:\Users\IanVeep\kdu.exe
  - C:\Users\IanVeep\infinity_hook_pro_max.sys
  - C:\Users\IanVeep\drv64.dll

  Registry Persistence

  - HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Run\KeePass - AutoRun persistence

  Network Indicators

  - Port 4443 - Meterpreter C2 (ESTABLISHED connection)
  - Port 4444 - Secondary C2 channel
  - Port 8000 - HTTP server for payload delivery

  User Account

  - AzureAD\IanVeep - Compromised user account